In [ ]:
import json
import urllib.parse
import boto3

def lambda_handler(event, context):
    print(event)
    s3 = boto3.client('s3')
    comprehend = boto3.client("comprehend")
    s3_writer = boto3.resource('s3')
    
    #print("Received event: " + json.dumps(event, indent=2))
    
    # Get the object from the event and show its content type
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = urllib.parse.unquote_plus(event['Records'][0]['s3']['object']['key'], encoding='utf-8')
    print(key)
    
    # Fetching latest file added to the bucket.
    objs = s3.list_objects_v2(Bucket=bucket, Prefix='Data/stream/')['Contents']
    sorted_objs = sorted(objs,key = lambda obj: obj['LastModified'],reverse=True)
    resent_file = s3.get_object(
                    Bucket=bucket,
                    Key=sorted_objs[0]['Key'])
     
    print(resent_file)
    
    # Reading file contents.
    content = resent_file['Body'].read().decode().rstrip('\n')
    content = content.split('\n')
    
    # Performing Tweets Sentiments and storing each tweet inside a list.
    transformed_tweets = []
    for tweet in content:
        
        tweet = json.loads(tweet)
        # Removing non ascii characters (emojis) from text 
        tweet_txt = tweet['text'].encode('ASCII','ignore').decode()
        # Getting text sentiment analysis
        sentiment = comprehend.detect_sentiment(Text=tweet_txt, LanguageCode="en")
        # Collecting tweet sentiment info into dictionary
        sentiment_tweet = {}
        sentiment_tweet['tweet_id'] = tweet['tweet_id']
        sentiment_tweet['Sentiment'] = sentiment['Sentiment']
        sentiment_tweet['Processed_text'] = tweet_txt
        
        transformed_tweets.append(sentiment_tweet)
    
    # Saving json data as string and adding new line character after each tweet.
    transformed_tweets = json.dumps(transformed_tweets)[1:-1].replace('},','}\n')
    # Renaming the sentiment file similar to the event file but with few name changes.
    output_file = key.replace('stream/','sentiment/').replace('Ingest','sentiment')
    # Writing sentiment file to S3 bucket.
    file_writer= s3_writer.Object(bucket, output_file)
    respond = file_writer.put(Body=transformed_tweets.encode())